# RAGAS (Retrieval Augmented Generation Assessment) - 精要摘要

一、整體概述

RAGAS（Retrieval-Augmented Generation Assessment）是一個專為 RAG 系統量身打造的評估與資料生成框架。
其核心價值在於：在不依賴大量人工標註的情況下，利用 LLM 作為評審（LLM-as-a-Judge），系統化地量化評估 RAG 的整體品質。


二、為何需要 RAGAS

傳統的 LLM 評估方式難以準確衡量 RAG 系統，主要原因包括：

- 回答品質同時受到 檢索結果 與 生成模型 的影響
- 人工評估成本高，且可重現性低
- 缺乏標準化指標來判斷「模型是否真的正確使用了檢索到的內容」

RAGAS 提供一套 自動化、模型驅動（LLM-based） 的評估解法，補足上述缺口。


三、核心評估指標（Metrics）

在評估 RAG 系統時，可將問題拆解為「檢索」與「生成」兩個主要維度，形成以下指標體系：

1. 檢索品質（Retrieval）

- 內容精準度（Context Precision）
  - 核心理念： 評估檢索系統的 「排序能力 (Ranking)」。

  - 優化說明： 這不僅衡量找得準不準，更看重相關片段是否被排在最前面。在 RAGAS 中，若相關內容排在第 1 名，得分會遠高於排在第 5 名，這直接影響了 LLM 在有限視窗內獲取資訊的效率。

- 內容召回率（Context Recall）
  - 核心理念： 評估檢索結果是否完整涵蓋了回答問題所需的所有事實。


2. 生成品質（Generation）

- Faithfulness（忠實度）
  - 核心理念： 確保答案中的每個事實都能在 Context 中找到依據（避免幻覺）。

- Answer Relevance（答案相關性）
  - 核心理念： 衡量答案的 「針對性 (Pertinence)」。
  - 優化說明： 該指標透過 LLM 將答案「反推」成問題，並計算與原問題的相似度。它評估的是答案是否直接回應了問題且不囉唆。注意： 即使答案包含事實錯誤（那是 Faithfulness 的事），只要它精準地針對問題回答，此項得分仍會很高。

- Answer Accuracy（答案正確性）
  當存在標準答案（Ground Truth）時，衡量生成結果與標準答案的一致程度。


3. 穩定性與抗噪能力（Advanced）

- Noise Sensitivity（噪音敏感度）
  - 核心理念： 評估模型在受干擾資訊影響下，是否仍能給出正確答案。
  - 優化說明： 此指標需區分兩種噪音：
      - 相關噪音 (Relevant Noise)： 內容與主題相關，但包含錯誤資訊或自相矛盾的事實。
      - 完全無關噪音 (Irrelevant Noise)： 檢索到了完全不相干的雜訊，模型是否能無視它們而非將其混入答案中。

四、合成資料生成（Synthetic Data Generation）

1. 核心理念

RAGAS 能夠從原始文件中自動產生以下資料，用於測試與回歸評估：

- 問題（Questions）
- 對應上下文（Contexts）
- 參考答案（Ground Truth）


2. 問題生成類型

- Single-hop Questions：僅需單一文件或單一事實即可回答
- Multi-hop Questions：需要結合多段資訊，或跨文件進行推理
- Reasoning-based Questions：涉及推論、比較或總結等高階認知能力


3. 知識圖譜導向設計

透過將文件轉換為 知識圖譜（Knowledge Graph），可進一步提升問題生成的多樣性與知識覆蓋率。


五、系統架構與整合方式

RAGAS 採用模組化設計，能輕鬆整合至不同 RAG 生態系，包括：

- LangChain
- LlamaIndex
- 自建的 RAG Pipeline

同時支援：

- 批次評估（Batch Evaluation）
- CI / CD 流程中的自動回歸測試
- 不同模型與檢索策略之間的橫向比較


## Quick Start

In [ ]:
import os

os.chdir("../")

In [ ]:
from ragas.embeddings import HuggingFaceEmbeddings
from ragas import evaluate, experiment

In [ ]:
# Embedding model
embeddings = HuggingFaceEmbeddings("BAAI/bge-m3")

### Ollama Cloud Service

雲端模型連結的方式已經寫好了，直接用就行。

In [ ]:
from src.ollama_connection import ragas_ollama

ragas_llm = ragas_ollama("gpt-oss:120b-cloud")

### RAGAS after v0.4 API

Mock Data:

In [ ]:
from ragas import Dataset

dataset = Dataset(name="test_dataset", backend="local/csv", root_dir="week_5")

test_data = [
    {"user_input": "什麼是 RAG?", 
     "response": "RAG 是檢索增強生成。", 
     "retrieved_contexts": ["RAG 結合了檢索與生成技術..."], 
     "reference": "RAG 是結合檢索與生成的技術。"},
]

for sample in test_data:
    dataset.append(sample)

# 如何儲存 dataset
dataset.save()

In [ ]:
# 讀取
Q = Dataset.load(name="test_dataset", root_dir="week_5", backend="local/csv",)

In [ ]:
len(Q)

建立實驗，計算 Faithfulness 和 AnswerRelevancy

In [ ]:
from pydantic import BaseModel
from ragas import experiment
from ragas.metrics.collections import Faithfulness, AnswerRelevancy

# Define experiment result structure
class ExperimentResult(BaseModel):
    faithfulness: float
    answer_relevancy: float


# Create experiment function
@experiment(ExperimentResult)
async def run_evaluation(row):
    faithfulness = Faithfulness(llm=ragas_llm)
    answer_relevancy = AnswerRelevancy(llm=ragas_llm, embeddings=embeddings)
    
    faith_result = await faithfulness.ascore(
        response=row.get("response"),
        retrieved_contexts=row.get("retrieved_contexts"),
        user_input=row.get("user_input")
    )

    relevancy_result = await answer_relevancy.ascore(
        user_input=row.get("user_input"),
        response=row.get("response")
    )

    return ExperimentResult(
        faithfulness=faith_result.value,
        answer_relevancy=relevancy_result.value
    )

# Run experiment
exp_results = await run_evaluation.arun(dataset)

In [ ]:
exp_results.to_pandas()

# RAGAS - 指標(metric)

## 內容精準度 (Context Precision)內容精準度 
(Context Precision) 是一項衡量指標，用於評估檢索器（Retriever）針對給定查詢時，將檢索內容中相關區塊（Relevant Chunks）的排名優於無關區塊的能力。具體而言，它評估了檢索內容中相關區塊被置於排名頂部的程度。其計算方式為內容中每個區塊的 Precision@k 平均值。Precision@k 是指在排名第 $k$ 位時，相關區塊數量與排名第 $k$ 位總區塊數量的比例。

$$
\text{Context Precision@K} =
\frac{\sum_{k=1}^{K} \left(\text{Precision@k} \times v_k\right)}
{\text{Total number of relevant items in the top } K \text{ results}}
$$


$$
\text{Precision@k} =
\frac{\text{true positives@k}}
{\text{true positives@k} + \text{false positives@k}}
$$

ContextPrecision 指標用來評估「檢索到的上下文是否對回答問題有幫助」。它會將每一段檢索到的上下文與參考答案（reference answer）進行比對，以判斷其相關性與實用性。當你有參考答案可用時，就適合使用這個指標。

In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.metrics.collections import ContextPrecision

# Create metric
scorer = ContextPrecision(llm=ragas_llm)

# Evaluate
result = await scorer.ascore(
    user_input="台北 101 在哪裡？",
    reference="台北 101 位於台灣台北市。",
    retrieved_contexts=[
        "台北 101 位於台灣台北市。", # relevant (v_1 = 1)
        "日月潭位於南投縣。" # irrelevant (v_2 = 0)
    ]
)
print(f"Context Precision Score: {result.value}")

In [ ]:
# Evaluate
result = await scorer.ascore(
    user_input="台北 101 在哪裡？",
    reference="台北 101 位於台灣台北市。",
    retrieved_contexts=[
        "日月潭位於南投縣。",
        "台北 101 位於台灣台北市。"
    ]
)
print(f"Context Precision Score: {result.value}")

## 回答相關性 (Answer Relevance)
當一個回答能直接且妥善地解決原始問題時，即被視為具有相關性。重要的是，我們對回答相關性的評估並不考慮真實性 (Factuality)，而是針對回答內容不完整或包含冗餘細節的情況進行扣分。

你可以想像成回答是否對於問題有足夠的針對性，可以避免出現雞同鴨講，或是回答是專業的幹話。

為了計算此分數，系統會引導大型語言模型（LLM）針對生成的回答多次產生對應的問題，並測量這些生成問題與原始問題之間的平均餘弦相似度 (Mean Cosine Similarity)。其核心理念在於：如果生成的回答能準確解決初始問題，那麼 LLM 應該能夠從該回答中反向生成與原始問題一致的問題。

$$
\text{Answer Relevancy} =
\frac{1}{N}
\sum_{i=1}^{N}
\text{cosine similarity}(E_{g_i}, E_o)
$$

為了計算答案與給定問題之間的相關性，我們採用兩個步驟：

### 步驟一
使用大型語言模型（LLM）從生成的答案中逆向推導出 **n** 個問題變體。  
例如，針對第一個答案，LLM 可能會生成以下可能的問題：

- 問題一：台灣位於亞洲的哪個區域？
- 問題二：台灣在亞洲中的地理位置是什麼？
- 問題三：你能指出台灣位於亞洲的哪個地區嗎？

### 步驟二
計算這些生成問題與實際問題之間的平均餘弦相似度。

### 核心概念
其核心概念在於：如果答案確實正確回應了問題，那麼僅根據該答案，就極有可能重建出原始問題。


In [ ]:
from ragas.metrics.collections import AnswerRelevancy

# Create metric
scorer = AnswerRelevancy(llm=ragas_llm, embeddings=embeddings)

# Evaluate
result = await scorer.ascore(
    user_input="台北 101 什麼時候完工？",
    response="台北 101 於 2004 年完工。"
)
print(f"Answer Relevancy Score: {result.value}")

## 忠實度（Faithfulness）

Faithfulness 指標用於衡量回應內容與所檢索上下文之間在事實上的一致性。其分數範圍為 0 到 1，分數越高代表一致性越好。

若回應中的所有主張都能由檢索到的上下文所支持，則該回應被視為是忠實的（faithful）。

計算方式如下：

1. 辨識回應中的所有主張。
2. 檢查每一項主張是否能從檢索到的上下文中推論得出。
3. 使用以下公式計算忠實度分數：

$$
\text{忠實度分數} = \frac{\text{回應中可由檢索上下文支持的主張數量}}{\text{回應中的主張總數}}
$$

In [ ]:
from ragas.metrics.collections import Faithfulness

# Create metric
scorer = Faithfulness(llm=ragas_llm)

# Evaluate
result = await scorer.ascore(
    user_input="台北 101 什麼時候完工？",
    response="台北 101 於 2004 年完工。",
    retrieved_contexts=[
        "台北 101 是台灣台北市的一棟摩天大樓，於 2004 年建成並正式開放使用。"
    ]
)
print(f"Faithfulness Score: {result.value}")

In [ ]:
result

有時候會因為缺乏精確的關鍵字而產生False Negative，導致Faithfulness下降

In [ ]:
# Evaluate
result = await scorer.ascore(
    user_input="蔣介石在哪裡出生？出生日期是什麼時候？",
    response="蔣介石出生於浙江紹興，出生日期是 1887 年 10 月 31 日。",
    retrieved_contexts=[
        "蔣中正（蔣介石，1887 年 10 月 31 日－1975 年 4 月 5 日）出生於浙江紹興，是中國近代政治與軍事領袖。"
    ]
)
print(f"Faithfulness Score: {result.value}")

## (Context Recall)

評估檢索到的 contexts 是否涵蓋了 reference 中的所有事實點

In [ ]:
from ragas.metrics.collections import ContextRecall

scorer = ContextRecall(llm=ragas_llm)

## 噪音敏感度（Noise Sensitivity）

Noise Sensitivity 越高 (接近 1)，代表模型越容易被無關資訊誤導。我們追求的是 低分（接近 0），但前提是 Faithfulness 也要高。

🔍 理解 RAGAS 中的噪音敏感度

噪音敏感度衡量生成模型（LLM）在面對檢索到的上下文中**不正確或不相關資訊**時受到干擾的程度。需要注意的是，根據原始碼的實作，這個指標是**具條件性的**，必須謹慎解讀。

---

邏輯運算（懲罰計算）

根據 RAGAS 的實作，答案中的某一項主張，只有在**同時滿足三個布林條件**時，才會觸發「噪音懲罰（Noise Penalty）」。

對於一個主張 $c$，懲罰公式如下：

$$
\text{Penalty}(c) = (\text{Context} \cap \text{GT}) \wedge (\text{Context} \cap \text{Answer}) \wedge \neg \text{GT}
$$

條件說明：

- $(\text{Context} \cap \text{GT})$：上下文必須是**相關的**（包含正確答案／事實）。
- $(\text{Context} \cap \text{Answer})$：模型必須對該上下文保持**忠實**（答案確實是從上下文中抽取的）。
- $\neg \text{GT}$：該主張必須是**錯誤的**（不存在於 Ground Truth 中）。

---

「零分」的細微差異（Zero Score Nuance）

分數為 **0** 並不一定表示系統表現完美。由於使用了 $\wedge$（AND）邏輯，只要鏈條中的**任一條件不成立**，最終分數就會是 0。

| 情況 | 分數 | 原因 |
| :--- | :--- | :--- |
| **檢索品質不佳** | 0 | 上下文不包含 Ground Truth，「相關性」條件為 False |
| **模型產生幻覺** | 0 | 答案不在上下文中，「忠實性」條件為 False |
| **模型表現完美** | 0 | 答案符合 Ground Truth，「錯誤性」條件為 False |

> ⚠️ 注意事項：  
> 噪音敏感度應與 **Context Recall（上下文召回率）** 及 **Faithfulness（忠實度）** 一併評估，避免將「0 分」誤判為正向結果。


In [ ]:
from ragas.metrics.collections import NoiseSensitivity

# Create metric
scorer = NoiseSensitivity(llm=ragas_llm)

In [ ]:
from textwrap import dedent


result = await scorer.ascore(
    user_input="什麼是牛頓第二運動定律？它在物理上代表什麼意義？",
    
    response=dedent("""\
    牛頓第二運動定律指出，物體所受到的力會影響其運動狀態，
    力越大，物體運動變化越明顯。這個定律常用來解釋物體的運動行為。
    """),
    
    reference=dedent("""\
    牛頓第二運動定律指出，作用在物體上的合力等於物體質量與加速度的乘積，
    可表示為 F = ma。此定律說明力是造成物體加速度的原因，
    並建立了力、質量與運動之間的定量關係，是經典力學的核心定律之一。
    """),
    
    retrieved_contexts=[
        # 相關且正確的內容
        "牛頓第二運動定律說明，物體的加速度與所受合力成正比，與物體的質量成反比。",
        
        "在國中與高中物理中，常用 F = ma 來分析直線運動問題。",
        
        # 與主題相關但層次不同（輕微雜訊）
        "牛頓第一運動定律描述在無外力作用下，物體將維持靜止或等速直線運動。",
        
        # 明顯的雜訊內容
        "能量守恆定律指出，在封閉系統中，能量不會憑空產生或消失。",
    ]
)

In [ ]:
result

## 答案準確度（Answer Accuracy）

答案準確度用於衡量模型回應與給定問題之**參考標準答案（Ground Truth）**之間的一致程度。  
此指標透過兩個獨立的「**LLM-as-a-Judge**」提示進行評估，每個提示都會回傳一個評分（0、2 或 4）。

接著，該指標會將評分轉換為 \([0, 1]\) 的區間，並對兩位評審的分數取平均值。  
分數越高，表示模型的回答越接近參考答案。

評分對應意義如下：

- **0** → 回應不正確，或未回答與參考答案相同的問題。
- **2** → 回應與參考答案**部分一致**。
- **4** → 回應與參考答案**完全一致**。

In [ ]:
from ragas.metrics.collections import AnswerAccuracy
# Create metric
scorer = AnswerAccuracy(llm=ragas_llm)

# Evaluate
result = await scorer.ascore(
    user_input="牛頓第二運動定律的數學表示式是什麼？",
    
    response="牛頓第二運動定律的數學表示式為 F = ma。",
    
    reference="牛頓第二運動定律指出，物體所受合力等於質量乘以加速度，其數學表示式為 F = ma。"
)

print(f"Answer Accuracy Score: {result.value}")

In [ ]:
result = await scorer.ascore(
    user_input="光在真空中的速度約為多少？",
    response="光在真空中的速度約為 3 × 10^6 公尺每秒。",
    reference="光在真空中的速度約為 3 × 10^8 公尺每秒。"
)

print(f"Answer Accuracy Score: {result.value}")

# RAGAS - 知識圖譜(Knowledge Graph, KG)創建

它是為了模擬人類「跨文本推理」的邏輯，從而生成高品質的 Multi-hop 測試題。

1. KG 的核心角色：模擬人類推理
在 RAGAS v0.4 中，構建 KG 不僅是為了存儲資料，更是為了模擬人類的 「跨文本推理 (Multi-hop Reasoning)」 邏輯。

- 三元組 (Subject-Predicate-Object)： 透過 Extractor 提取出的三元組，定義了節點間的顯性聯繫。
- 測試題生成 (Testset Generation)： 利用 KG 的連結性，RAGAS 可以自動規劃「跨節點路徑」。例如：問題需要從節點 A 跳轉到節點 B 才能獲得完整答案，從而生成難度更高的 Multi-hop 測試題，這比單純的語意搜尋測試更具挑戰性。

## 什麼是圖(Graph)

圖的基本構成是
1. 節點(node)
2. 邊(edge): 定義了節點之間的邏輯連結。在KnowledgeGraph中，邊被稱為Relationship

最簡單的圖由兩個節點和一個Relationship構成:

1. 組成要素
- 節點 A (Node A)：領主 (Lord)
- 節點 B (Node B)：騎士 (Knight)
- 關係 (Relationship)：效忠契約 (Pledges Fealty)

2. 圖的結構
- 這是一個最基礎的「三元組」（Triple），在知識圖譜中通常表示為 (主體) --[關係]--> (客體)：
  
    [騎士] -- (效忠於) --> [領主]

RAGAS的知識圖產生由三種模塊所組成:
1. 節點生成
2. 節點上的特徵抽取，例如embeddings, Summary, NER(Named-entity recognition), KeyPhrases, Headlines，等等
3. 建立Relationship: 節點之間的相似度計算

![caption](knowledge_graph.jpg)

## 官方範例
我們將探討 Ragas 的測試集生成模組，用以為基於 **檢索增強生成（Retrieval-Augmented Generation, RAG）** 的問答機器人創建合成測試集。我們的目標是設計一個 **Ragas 航空助理**，能夠回答客戶在各種主題上的問題，包括： 

- 航班預訂 (Flight booking)
- 航班變更與取消  (Flight changes and cancellations_
- 行李政策  (Baggage policies)
- 查看預訂資訊 (Viewing reservations)
- 航班延誤 (Flight delays)
- 機上服務  (In-flight services)
- 特殊協助 (Special assistance)
 
為了確保我們的合成數據集盡可能真實且多樣化，我們將創建不同的客戶角色（customer personas）。每個角色將代表不同類型的旅客及其行為，有助於我們構建一個全面且具代表性的測試集。這種方法確保我們能夠徹底評估 RAG 模型的效果與穩健性。

**來源:** [Ragas 官方文件](https://docs.ragas.io/en/v0.4.0/howtos/applications/singlehop_testset_gen/)

```bash
git clone https://huggingface.co/datasets/vibrantlabsai/ragas-airline-dataset week_5/ragas-airline-dataset
```

這是英文資料，可以照著RAGAS官方給出的範例一步步地進行。

但用中文還是比較有挑戰性的，沒有挑戰性的東西就不好玩了。

In [ ]:
!pip install opencc-python-reimplemented

用 opencc-python-reimplemented 將文字統一轉為台灣繁體。

In [ ]:
from pathlib import Path

import wikipediaapi
from opencc import OpenCC

cc = OpenCC('s2twp')

wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='zh-tw')

wiki_titles = ["書劍恩仇錄", "碧血劍", "射鵰英雄傳", "雪山飛狐", "神鵰俠侶", "飛狐外傳", "白馬嘯西風", "鴛鴦刀",
               "倚天屠龍記", "連城訣", "天龍八部", "俠客行", "笑傲江湖", "鹿鼎記", "越女劍"]

for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = cc.convert(page.text)

    data_path = Path("week_5/data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

path = "week_5/data"
loader = DirectoryLoader(path, glob="**/*.txt")
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
import os

os.chdir("../")

from ragas.embeddings import HuggingFaceEmbeddings

from src.ollama_connection import ragas_ollama

ragas_llm = ragas_ollama(model='gpt-oss:120b-cloud')

embeddings = HuggingFaceEmbeddings("BAAI/bge-m3")

In [ ]:
docs[0].metadata

### 創建 Nodes

快速建立一個Knowledge Graph.其中的Node只包含了原始數據，沒有其他的特徵和關聯。

某些方面來說，知識圖的建立跟Index相同，都是基於節點的設計。

In [ ]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType

single_hop_kg = KnowledgeGraph()

for doc in docs:
    single_hop_kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

single_hop_kg

### 節點特徵提取

不同的擷取器用於從每個節點中提取可用於建立節點間關聯的信息。例如，針對財務文件，可以使用的擷取器包括：

- **實體擷取器 (Entity Extractor)：** 提取實體，如公司名稱。  
- **關鍵詞擷取器 (Keyphrase Extractor)：** 提取每個節點中重要的關鍵詞或短語。  

你也可以撰寫自訂擷取器，以提取對你的領域相關的重要資訊。

> 擷取器可以基於大型語言模型（LLM Based），繼承自 `LLMBasedExtractor`；或者基於規則（Rule Based），繼承自 `Extractor`。

若是不知道如何取得import路徑 (並不是所有的documentation都會寫得很詳細)，可以參照以下範例

In [ ]:
import inspect

from ragas.testset.transforms import HeadlinesExtractor

module = inspect.getmodule(HeadlinesExtractor.prompt)
print(module.__name__) 
print(module.__file__)

In [ ]:
# HeadlinesExtractor?

1. HeadlinesExtractor

In [ ]:
from textwrap import dedent

from ragas.testset.transforms.extractors.llm_based import HeadlinesExtractorPrompt, TextWithExtractionLimit, Headlines
# 透過控制 HeadlinesExtractor.prompt來改變行

headline_extractor_prompt = HeadlinesExtractorPrompt(language="traditional chineses")
headline_extractor_prompt.instruction = dedent("""\
    從給定的文本中擷取最重要、最多 max_num 個可用作切分文本為獨立段落的標題。
    請專注於第二層與第三層標題（Level 2 與 Level 3 標題），確保這些標題能清楚界定各段內容的主題與邊界。
    確保輸出語言為繁體中文(traditional Chinese)
    """)

text_with_extraction_limit = TextWithExtractionLimit(text=dedent("""\
                前言
                主題的整體概述……

                核心概念
                對主要概念的說明……

                詳細分析
                分析所使用的技術與方法……

                子章節：專門技術
                對特定技術的進一步說明……

                未來發展方向
                對未來趨勢的洞見……

                子章節：研究的下一步
                對新研究領域的討論……

                結論
                最終總結與結語。
                """), max_num=6)

headlines = Headlines(headlines=['前言', '核心概念', '詳細分析', '子章節：專門技術', '未來發展方向', '結論'])

headline_extractor_prompt.examples = [(text_with_extraction_limit, headlines)]

headline_extractor = HeadlinesExtractor(llm=ragas_llm, max_num=50, prompt=headline_extractor_prompt)

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

output = [await headline_extractor.extract(node) for node in sample_nodes]

將抽取出來的特徵放入node中

In [ ]:
_ = [node.properties.update({key:val}) for (key,val), node in zip(output, sample_nodes)]

2. KeyPhrasesExtractor

In [ ]:
from ragas.testset.transforms import KeyphrasesExtractor

# KeyphrasesExtractor?

In [ ]:
from ragas.testset.transforms import KeyphrasesExtractor
from ragas.testset.transforms.extractors.llm_based import KeyphrasesExtractorPrompt, TextWithExtractionLimit, Keyphrases

keyphrases_extractor_prompt = KeyphrasesExtractorPrompt(language="繁體中文/traditional Chinese")
keyphrases_extractor_prompt.instruction = "從給定文字中抽取前 max_num 個關鍵詞。確保輸出語言為繁體中文(traditional Chinese)"

text_with_extraction_limit = TextWithExtractionLimit(text=dedent("""\
                人工智慧

                人工智慧正在改變各行各業，透過自動化原本需要人類智慧的任務來提高效率。從醫療到金融，AI 被用來快速且準確地分析大量資料。這項技術也推動了自駕車和個人化推薦等領域的創新。
                """), max_num=5)

key_phrases = Keyphrases(keyphrases =["人工智慧", "自動化任務", "醫療", "自駕車", "個人化推薦"])

keyphrases_extractor_prompt.examples = [(text_with_extraction_limit, key_phrases)]

keyphrases_extractor = KeyphrasesExtractor(llm=ragas_llm, max_num=30, prompt=keyphrases_extractor_prompt)

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

output = [await keyphrases_extractor.extract(node) for node in sample_nodes]

3. EmbeddingExtractor

提取embeddings

In [ ]:
from ragas.testset.transforms.extractors import EmbeddingExtractor

embedding_extractor = EmbeddingExtractor(
    embedding_model=embeddings,
    embed_property_name="page_content",     # text source
    property_name="embedding" # where vectors are stored
)

4. SummaryExtractor

In [ ]:
from ragas.testset.transforms.extractors import SummaryExtractor

SummaryExtractor?

In [ ]:
from ragas.testset.transforms.extractors.llm_based import SummaryExtractorPrompt
from ragas.testset.transforms.extractors.llm_based import StringIO

summary_extractor_prompt = SummaryExtractorPrompt(language="繁體中文/traditional Chinese")
summary_extractor_prompt.instruction = "將給定的文字總結為不超過 10 句話。使用繁體中文(traditional Chinese)回應"

summary_extractor_prompt = SummaryExtractorPrompt(language='繁體中文/traditional Chinese')

examples = [(StringIO(text=dedent("""
                                  人工智慧
                
                                  人工智慧正在改變各行各業，透過自動化原本需要人類智慧的任務來提高效率。從醫療到金融，AI 被用來快速且準確地分析大量資料。
                                  這項技術也推動了自駕車和個人化推薦等領域的創新。
                                  確保輸出語言為繁體中文(traditional Chinese)
                                  """)),
             StringIO(text=dedent("""
                                  人工智慧正在改變各行各業，透過自動化任務、分析資料並推動創新，如自駕車和個人化推薦。
                                  """)))]

summary_extractor_prompt.examples = examples

In [ ]:
summary_extractor = SummaryExtractor(
    llm=ragas_llm,
    property_name="summary", # where vectors are stored
    prompt=summary_extractor_prompt
)

In [ ]:
_ = [node.properties.update({key:val}) for (key,val), node in zip(output, sample_nodes)]

## 關聯建立器 (Relationship Builder)

擷取到的信息用於建立節點之間的關聯。例如，在財務文件的情境中，可以根據節點中出現的實體來建立節點之間的關係。  

你也可以撰寫自訂的關聯建立器（Custom Relationship Builder），根據對你的領域相關的信息來建立節點間的關聯。

1. JaccardSimilarityBuilder

Jaccard Similarity:

**Jaccard 相似度**用來衡量兩個集合之間的相似程度，定義為兩個集合「交集」大小與「聯集」大小的比值。

$$
\text{Jaccard Similarity}(A, B) = \frac{|A \cap B|}{|A \cup B|}
$$

- 數值範圍介於 **0 到 1**  
- **0** 代表完全沒有重疊，**1** 代表兩個集合完全相同  
- 常用於文字分析、推薦系統與分群（clustering）等應用場景

In [ ]:
from ragas.testset.transforms.relationship_builders.traditional import JaccardSimilarityBuilder

# 根據 headlines這個property，基於Jaccard Similarity建立起節點之間的連結
# 必須先確保所有 Node 都已經透過某個Extractors處理過，並且有得到headlines這個property

rel_builder = JaccardSimilarityBuilder(property_name="headlines", new_property_name="entity_jaccard_similarity")

2. ConsineSimilarityBuilder

根據cosine相似度來決定節點之間的距離。

In [ ]:
from ragas.testset.transforms.relationship_builders.cosine import CosineSimilarityBuilder

cosine_relationship_builder = CosineSimilarityBuilder(
    property_name="embedding",     # CosineSimilarityBuildernode attribute containing embeddings
    threshold=0.75,                # cosine similarity threshold
)

3. SummaryCosineSimilarityBuilder

根據page content抽取Summary。跟SummaryIndex相似。然後根據Summary之間的cosine similarity建立起node之間的關聯

這個可以當成是 CosineSimilarityBuilder的特例

In [ ]:
from ragas.testset.transforms.relationship_builders.cosine import SummaryCosineSimilarityBuilder

summary_cosine_relationship_builder = SummaryCosineSimilarityBuilder(property_name='summary_embedding')

4. OverlapScoreBuilder（重疊分數建構器）

OverlapScoreBuilder 是 RAGAS 知識圖中非常關鍵的一個 Builder。
它產生的 relationship property，會在 Multihop 數據合成 時，決定「哪些節點彼此可以一起用來合成資料」。

換句話說：
👉 它負責判斷兩個 node 之間是否「夠像」，值得被連起來。

---

核心設定參數（從程式碼看）

```python
property_name = "entities"
new_property_name = "overlap_score"
distance_measure = JARO_WINKLER
distance_threshold = 0.9
threshold = 0.01
```

- property_name：用來比較的節點屬性（例如 entities）
- distance_measure：文字距離計算方式（Jaro-Winkler）
- distance_threshold：單一文字是否視為「重疊」的相似度門檻
- threshold：整體重疊比例的最低門檻

運作流程（邏輯拆解

一、文字層級的相似度判斷（item-level）

1. 對 node X 與 node Y 中的每一個文字特徵進行比對：
    - 使用距離函數計算文字相似度
2. 若 similarity >= distance_threshold：
    - 視為「重疊」（記為 1）
    - 將這組 (x, y) 存入 overlapped_items
    - 否則視為不重疊（記為 0）

```python

overlaps = []
overlapped_items = []
for x in node_x_items:
    if x not in noisy_items:
        for y in node_y_items:
            if y not in noisy_items:
                similarity = 1 - distance_measure.distance(
                    x.lower(), y.lower()
                )
                verdict = similarity >= self.distance_threshold
                overlaps.append(verdict)
                if verdict:
                    overlapped_items.append((x, y))
```

二、節點層級的重疊判斷（node-level）

1. 根據 overlaps 計算整體 overlap score
2. 若 overlap_score >= threshold：
    - 在兩個 nodes 之間建立 relationship

```pytnon
similarity = self._overlap_score(overlaps)
if similarity >= self.threshold:
    relationships.append(
        Relationship(
            source=node_x,
            target=node_y,
            type=f"{self.property_name}_overlap",
            properties={
                f"{self.property_name}_{self.new_property_name}": similarity,
                "overlapped_items": overlapped_items,
            },
        )
    )
```

產生的 Relationship 結構

- type
    - 由 property_name 決定
    - 例如：entities_overlap
- properties
    - entities_overlap_score：整體重疊比例
    - overlapped_items：實際被判定為重疊的文字配對清單

在下一周的Multihop數據合成中，你會看到

- `type`
- `overlapped_items`

扮演著不可或缺的地位

In [ ]:
from ragas.testset.transforms import OverlapScoreBuilder 

overlap_relationship_builder = OverlapScoreBuilder(property_name="keyphrases", new_property_name="overlap_score",
                                                   threshold=0.01, distance_threshold=0.5)

## Transforms

我們可以將之前的提取器(Extractors)和連結器(Relationship Builders)進行並聯/串聯來產生轉換流

### 最小範例

1. 建立兩個原始節點
2. 提取headlines
3. 根據headlines分割文件，並產生新的nodes

In [ ]:
from ragas.testset.transforms import HeadlineSplitter, apply_transforms
from ragas.testset.graph import KnowledgeGraph, Node, NodeType

headline_splitter = HeadlineSplitter(max_tokens=500)

"""
headline_splitter的作用是將 page_content 根據 headlines 的位置進行拆解，
並且產生兩種新的nodes: `child` (從大nodes中過來) 和 `next` (各個小nodes之間的聯繫)
"""

sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

kg = KnowledgeGraph(nodes=sample_nodes)

transforms_basic = [
    headline_extractor,
    headline_splitter
]

apply_transforms(kg, transforms=transforms_basic)

In [ ]:
# kg.relationships

In [ ]:
# kg.nodes

### 加入 SummaryEmbedding

下一步:
1. 提取 summary
2. 提取 summary embeddings

In [ ]:
from ragas.testset.transforms.extractors import EmbeddingExtractor
from ragas.run_config import RunConfig

run_config = RunConfig(
    max_workers=3,   # 👈 控制同時 LLM 請求數
    timeout=120,
    max_retries=5
)

summary_embedding_extractor = EmbeddingExtractor(
    embedding_model=embeddings,
    embed_property_name="summary",     # text source
    property_name="summary_embedding" # where vectors are stored
)

transforms_summary = [
    summary_extractor,
    summary_embedding_extractor
]

apply_transforms(kg, transforms=transforms_summary, run_config=run_config)

In [ ]:
kg.nodes[:10]

In [ ]:
kg.nodes[:10][2].properties.keys()

繼續...

1. 提取keyphrases
2. 使用overlap builder

In [ ]:
from ragas.testset.transforms import OverlapScoreBuilder 

overlap_relationship_builder = OverlapScoreBuilder(property_name="keyphrases", new_property_name="overlap_score",
                                                   threshold=0.25, distance_threshold=0.8)


transforms_keyphrases_overlap = [keyphrases_extractor,
                                 overlap_relationship_builder]

apply_transforms(kg, transforms=transforms_keyphrases_overlap, run_config=run_config)

上述的流程可以用以下的表格表示:

| 順序 | 轉換組件（Transform） | 目的 |
|----|---------------------|------|
| 1 | HeadlinesExtractor | 找出文件的結構化標題 |
| 2 | HeadlineSplitter | **關鍵**：將大文件切成具備父子關係的小 Node |
| 3 | KeyphrasesExtractor | 為小 Node 提取關鍵詞 |
| 4 | EmbeddingExtractor | 向量化，用於語義找尋 |
| 5 | OverlapScoreBuilder | **關鍵**：根據關鍵詞重疊度連結不同 Node（建立邊） |

根據relationship的 type 抽取 source-relationship-target 三元祖

In [ ]:
results_keyphrases_overlap = kg.find_two_nodes_single_rel(
            relationship_condition=lambda rel: (
                True if rel.type == "keyphrases_overlap" else False
            )
        )

In [ ]:
results_keyphrases_overlap

Persist 知識圖

In [ ]:
kg.save("week_5/knowledge_graph_test.json")

讀取知識圖

In [ ]:
kg_load = KnowledgeGraph()

kg_load.load("week_5/knowledge_graph_test.json")